In [ ]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

# ALGORITHM 1 TESTING


Let's create two graphs from the list of edges downloaded from the Snap database. 

In [ ]:
G1 = nx.read_edgelist('../data/web-Stanford.txt', create_using=nx.DiGraph(), nodetype=int)

G2 = nx.read_edgelist('../data/web-BerkStan.txt', create_using=nx.DiGraph(), nodetype=int)

Creating the transition probability matrix

In [ ]:
# square matrix of size n x n, where n is the number of nodes in the graph. The matrix is filled with zeros and the (i,j) element is x if the node i is connected to the node j. Where x is 1/(number of nodes connected to i).

def create_matrix(G):
    n = G.number_of_nodes()
    P = sp.sparse.lil_matrix((n,n))
    for i in G.nodes():
        for j in G[i]: #G[i] is the list of nodes connected to i, it's neighbors
            P[i-1,j-1] = 1/len(G[i])
    return P

To ensure that the random process has a unique stationary distribution and it will not stagnate, the transition matrix P is usually modified to be an irreducible stochastic matrix A (called the Google matrix) as follows

$$ A = \alpha \tilde{P} + (1-\alpha)v e^T$$

Where $\tilde{P}$ is defined as 

$$ \tilde{P} = P + v d^T$$

Where $d \in \mathbb{N}^{n \times 1}$ s a binary vector tracing the indices of dangling web-pages with no hyperlinks, i.e., $d(i ) = 1$ if the `ith` page has no hyperlink, $v \in \mathbb{R}^{n \times 1}$ is a probability vector, $e = [1, 1, . . . , 1]^T$ , and $0 < \alpha < 1$ is the so-called damping factor that represents the probability in the model that the surfer transfer by clicking a hyperlink rather than other ways

In [ ]:
n = G1.number_of_nodes()
P = create_matrix(G1)    

the vector `d` solves the dangling nodes problem

In [ ]:
# define d as a nx1 sparse matrix, where n is the number of nodes in the graph. The vector is filled with d(i) = 1 if the i row of the matrix P is filled with zeros, other wise is 0

# d is the vector of dangling nodes
d = sp.sparse.lil_matrix((n,1))
for i in range(n):
    if P[i].sum() == 0:
        d[i] = 1

The vector v is a probability vector, the sum of its elements bust be one

In [ ]:
# define v as the probability vector of size n x 1, where n is the number of nodes in the graph. The vector is filled with 1/n

v = sp.sparse.lil_matrix((n,1))
for i in range(n):
    v[i] = 1/n  

# maybe I could have done it with a lambda more concisely? idk

Now we can compute the transition matrix


In [ ]:
Pt = P + v.dot(d.T)

# Pt is a sparse matrix too

In [ ]:
# e is a nx1 sparse matrix filled with ones
e = sp.sparse.lil_matrix((1,n))
for i in range(n):
    e[0,i] = 1

In [ ]:
# # v*eT is a nxn sparse matrix filled all with 1/n, let's call it B

# B = sp.sparse.lil_matrix((n,n))
# for i in range(n):
#     for j in range(n):
#         B[i,j] = 1/n

# A = alpha*Pt + (1-alpha)*B

Algorithm 1 Shifted-Power method for PageRank with multiple damping factors:

In [ ]:
# pandas dataframe to store the results
df = pd.DataFrame(columns=['alpha', 'iterations', 'tau', 'time'])

In [ ]:
def Algorithm1(Pt, v, tau, max_mv, a: list):
    
    start_time = time.time()

    u = Pt.dot(v) - v 
    mv = 1 # number of matrix vector products
    r = sp.sparse.lil_matrix((n,1)) 
    Res = sp.sparse.lil_matrix((len(a),1))
    x = sp.sparse.lil_matrix((n,1))  

    for i in range(len(a)):
        r = a[i]*(u) 
        normed_r = norm(r)
        Res[i] = normed_r 

        if Res[i] > tau:
            x = r + v 

    while max(Res) > tau and mv < max_mv:
        u = Pt*u 
        mv += 1 

        for i in range(len(a)):
            if Res[i] >= tau: 
                r = (a[i]**(mv+1))*(u)
                Res[i] = norm(r)

                if Res[i] > tau:
                    x = r + x

    if mv == max_mv:
        print("The algorithm didn't converge in ", max_mv, " iterations")
    else:
        print("The algorithm converged in ", mv, " iterations")

    total_time = time.time() - start_time
    print("The algorithm took ", total_time, " seconds")
       
    return mv, x, r, total_time  

Testing setup

In [ ]:
# list of alpha values, from 0.85 to 0.99 with step 0.01
a = []
for i in range(85,100):
    a.append(i/100)

max_mv = 1000

# run the algorithm for different values of tau from 10^-5 to 10^-9 with step 10^-1
for i in range(5,10):
    tau = 10**(-i)
    print("\ntau = ", tau)
    mv, x, r, total_time = Algorithm1(Pt, v, tau, max_mv, a)
    df = df.append({'alpha': a, 'iterations': mv, 'tau': tau, 'time': total_time}, ignore_index=True) 

In [ ]:
# save the results in a csv file
df.to_csv('../data/results/algo1/different_tau.csv', index=False)

Plotting the results of the algorithm for different values of tau, and fixed alpha

In [ ]:
x = df['tau'][::-1].tolist()
y = df['products m-v'].tolist()

fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers'),
                            layout=go.Layout(title='products needed for the convergence', xaxis_title='tau', yaxis_title='products matrix vector'))

# save the figure as a html file
fig.write_html("../data/results/algo1/taus_over_prods.html")
print("The plot has been saved in the folder data/results/algo1")

##### RESULTS OVER TIME #####

x1 = df['tau'][::-1].tolist()
y1 = df['time'].tolist()

fig = go.Figure(data=go.Scatter(x=x1, y=y1, mode='lines+markers'),
                                layout=go.Layout(title='Time needed for the convergence', xaxis_title='tau', yaxis_title='time (seconds)'))

# save the plot in a html file
fig.write_html("../data/results/algo1/taus_over_time.html")
print("The plot has been saved in the folder data/results/algo1")
